In [1]:
!pip install gradio transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [20]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

# Dictionary of available models
model_configs = {
    "distilbert-base-uncased": {"num_labels": 2, "output_attentions": True},
    "bert-base-uncased": {"num_labels": 2, "output_attentions": True},
    "roberta-base": {"num_labels": 2, "output_attentions": True}
}

# Global model and tokenizer storage
model = None
tokenizer = None

# Function to load or switch models
def load_model(model_name):
    global model, tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, **model_configs[model_name])
    return model_name

# Function to tokenize input text and get attention weights
def get_token_attributions(input_text, model_name):
    global model, tokenizer
    if model is None or model_name != model.config._name_or_path:
        load_model(model_name)

    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # Get model outputs with attention weights
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)

    # Extract attention weights (handle different model structures)
    attentions = outputs.attentions
    if attentions is not None:
        avg_attention = torch.stack(attentions).mean(dim=0).mean(dim=1)[0]  # [seq_len, seq_len]
        token_scores = avg_attention.diagonal().numpy()  # Self-attention scores
    else:
        # Fallback for models without direct attention output
        token_scores = np.ones(len(tokens))  # Placeholder scores
        print("Attention weights not available for this model. Using uniform scores.")

    # Normalize scores for visualization
    token_scores = (token_scores - token_scores.min()) / (token_scores.max() - token_scores.min())

    # Get model prediction (logits)
    logits = outputs.logits
    prediction = torch.softmax(logits, dim=-1).numpy()[0]
    label = "Positive" if prediction[1] > prediction[0] else "Negative"

    return tokens, token_scores, label, prediction

# Function to generate HTML for enhanced visualization
def generate_visualization_html(tokens, scores, emphasis=1.0):
    html = "<div style='font-family: Arial; padding: 10px;'>"
    html += "<h3>Token Attribution Visualization</h3>"
    html += "<p>Adjust the emphasis slider to change color intensity. Hover over tokens for details.</p>"

    # Color legend
    html += """
    <div style='margin-bottom: 10px;'>
        <strong>Legend:</strong>
        <div style='display: flex; align-items: center; gap: 5px;'>
            <div style='width: 100px; height: 10px; background: linear-gradient(to right, #e0ffe0, #006400);'></div>
            <span>Low Importance → High Importance</span>
        </div>
    </div>
    """

    html += "<div style='display: flex; flex-wrap: wrap; gap: 5px;'>"

    for i, (token, score) in enumerate(zip(tokens, scores)):
        emphasized_score = min(1.0, score * emphasis)
        r = int(224 * (1 - emphasized_score))
        g = int(255 * emphasized_score)
        bg_color = f"rgba({r}, {g}, 0, 0.5)"
        html += f"""
        <div style='background-color: {bg_color}; padding: 5px; border-radius: 3px; position: relative;'
             onmouseover='this.children[0].style.display="block"'
             onmouseout='this.children[0].style.display="none"'>
            {token}
            <div style='display: none; position: absolute; background-color: #333; color: white; padding: 5px; border-radius: 3px; z-index: 10;'>
                Token: {token}<br>Position: {i}<br>Score: {score:.3f}<br>Emphasized: {emphasized_score:.3f}
            </div>
        </div>
        """
    html += "</div>"
    return html

# Main function for Gradio interface with model selection
def analyze_text(input_text, model_name, emphasis):
    if not input_text.strip():
        return "Please enter some text.", "", "", gr.update(value=1.0), model_name

    # Get tokens, scores, and prediction
    tokens, scores, label, prediction = get_token_attributions(input_text, model_name)

    # Generate visualization with emphasis
    viz_html = generate_visualization_html(tokens, scores, emphasis)

    # Format prediction output
    pred_output = f"Model Prediction: {label} (Positive: {prediction[1]:.3f}, Negative: {prediction[0]:.3f})"

    return viz_html, pred_output, "Visualization generated. Hover over tokens to see details.", gr.update(value=emphasis), model_name

# Gradio interface
iface = gr.Interface(
    fn=analyze_text,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter text here..."),
        gr.Dropdown(choices=list(model_configs.keys()), value="distilbert-base-uncased", label="Select Model"),
        gr.Slider(minimum=0.5, maximum=2.0, value=1.0, label="Emphasis on Attention Scores")
    ],
    outputs=[
        gr.HTML(label="Token Attribution Visualization"),
        gr.Text(label="Model Prediction"),
        gr.Text(label="Status"),
        gr.Slider(minimum=0.5, maximum=2.0, value=1.0, label="Emphasis on Attention Scores"),
        gr.Dropdown(choices=list(model_configs.keys()), value="distilbert-base-uncased", label="Select Model")
    ],
    title="Token Attribution Visualization Tool",
    description="Enter text to visualize token-level importance using attention weights. Select a model and adjust the emphasis slider to modify color intensity. Hover over tokens for details."
)

# Launch the interface (Colab-compatible)
if __name__ == "__main__":
    iface.launch(debug=False)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://82f1371f8e2052509b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
